Nama : Indira Septianita Larasati
NIM : H071191023


In [3]:
import tensorflow as tf
import numpy as np

# Layers and Blocks 

In [4]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation=tf.nn.relu),
    tf.keras.layers.Dense(10),
])

X = tf.random.uniform((2, 20))
net(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[-0.08813918,  0.30137488, -0.09336432,  0.262882  ,  0.05273577,
        -0.02207185,  0.42664248, -0.1352494 ,  0.04247363,  0.26926205],
       [-0.1350706 ,  0.28126317, -0.1379056 ,  0.1255644 , -0.00947122,
         0.01033582,  0.47339958, -0.01137753, -0.09338392,  0.36239195]],
      dtype=float32)>

## A Custom Block

In [7]:
class MLP(tf.keras.Model):
   def __init__(self):
        super().__init__() 
        self.hidden = tf.keras.layers.Dense(units=256, activation=tf.nn.relu)
        self.out = tf.keras.layers.Dense(units=10)
   def call(self, X):
        return self.out(self.hidden((X)))


In [8]:
net = MLP()
net(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[-0.06802206, -0.16734335, -0.03420285,  0.04225944,  0.05599046,
         0.16531329,  0.01562939,  0.55300784, -0.5800906 ,  0.18123263],
       [-0.16980869, -0.05972652, -0.00710226, -0.14939418,  0.06984033,
         0.20337279,  0.03561009,  0.4177763 , -0.37610966,  0.01246594]],
      dtype=float32)>

## The Sequential Block

In [10]:
class MySequential(tf.keras.Model):
    def __init__(self, *args):
        super().__init__()
        self.modules = []
        for block in args:
            self.modules.append(block)

    def call(self, X):
        for module in self.modules:
            X = module(X)
        return X

In [11]:
net = MySequential(
    tf.keras.layers.Dense(units=256, activation=tf.nn.relu),
    tf.keras.layers.Dense(10))
net(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[-0.0520504 , -0.30513036, -0.12514259,  0.21755835, -0.13055296,
        -0.17406034,  0.1919541 ,  0.12196434, -0.09001108, -0.1981172 ],
       [ 0.12685545, -0.40223154,  0.06866871,  0.11975428, -0.20567635,
        -0.10210697,  0.24530908,  0.22550991, -0.28451934, -0.22377697]],
      dtype=float32)>

## Executing Code in the Forward Propagation Function

In [12]:
class FixedHiddenMLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()

        self.rand_weight = tf.constant(tf.random.uniform((20, 20)))
        self.dense = tf.keras.layers.Dense(20, activation=tf.nn.relu)

    def call(self, inputs):
        X = self.flatten(inputs)

        X = tf.nn.relu(tf.matmul(X, self.rand_weight) + 1)

        X = self.dense(X)
        
        while tf.reduce_sum(tf.math.abs(X)) > 1:
            X /= 2
        return tf.reduce_sum(X)


In [13]:
net = FixedHiddenMLP()
net(X)

<tf.Tensor: shape=(), dtype=float32, numpy=0.8253869>

In [14]:
class NestMLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.net = tf.keras.Sequential()
        self.net.add(tf.keras.layers.Dense(64, activation=tf.nn.relu))
        self.net.add(tf.keras.layers.Dense(32, activation=tf.nn.relu))
        self.dense = tf.keras.layers.Dense(16, activation=tf.nn.relu)

    def call(self, inputs):
        return self.dense(self.net(inputs))

chimera = tf.keras.Sequential()
chimera.add(NestMLP())
chimera.add(tf.keras.layers.Dense(20))
chimera.add(FixedHiddenMLP())
chimera(X)

<tf.Tensor: shape=(), dtype=float32, numpy=0.62708306>

# Parameter Management

In [15]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(4, activation=tf.nn.relu),
    tf.keras.layers.Dense(1),
])

X = tf.random.uniform((2, 4))
net(X)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[-0.22600053],
       [-0.4144465 ]], dtype=float32)>

## Parameter Access

In [16]:
print(net.layers[2].weights)

[<tf.Variable 'dense_15/kernel:0' shape=(4, 1) dtype=float32, numpy=
array([[ 1.0855875 ],
       [-0.73870146],
       [-0.34977245],
       [-0.49576533]], dtype=float32)>, <tf.Variable 'dense_15/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]


## Targeted Parameters

In [17]:
print(type(net.layers[2].weights[1]))
print(net.layers[2].weights[1])
print(tf.convert_to_tensor(net.layers[2].weights[1]))

<class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'>
<tf.Variable 'dense_15/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>
tf.Tensor([0.], shape=(1,), dtype=float32)


## All Parameters at Once

In [18]:
print(net.layers[1].weights)
print(net.get_weights())

[<tf.Variable 'dense_14/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[-0.0452475 ,  0.3680504 , -0.52012867, -0.50537884],
       [ 0.14687496,  0.20760447, -0.06339884,  0.7312334 ],
       [-0.46167246,  0.47644752, -0.3190418 , -0.50481635],
       [-0.75651395, -0.03857207,  0.7162841 ,  0.5960972 ]],
      dtype=float32)>, <tf.Variable 'dense_14/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>]
[array([[-0.0452475 ,  0.3680504 , -0.52012867, -0.50537884],
       [ 0.14687496,  0.20760447, -0.06339884,  0.7312334 ],
       [-0.46167246,  0.47644752, -0.3190418 , -0.50481635],
       [-0.75651395, -0.03857207,  0.7162841 ,  0.5960972 ]],
      dtype=float32), array([0., 0., 0., 0.], dtype=float32), array([[ 1.0855875 ],
       [-0.73870146],
       [-0.34977245],
       [-0.49576533]], dtype=float32), array([0.], dtype=float32)]


In [19]:
net.get_weights()[1]

array([0., 0., 0., 0.], dtype=float32)

## Collecting Parameters from Nested Blocks

In [20]:
def block1(name):
    return tf.keras.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(4, activation=tf.nn.relu)],
        name=name)

def block2():
    net = tf.keras.Sequential()
    for i in range(4):
        
        net.add(block1(name=f'block-{i}'))
    return net

rgnet = tf.keras.Sequential()
rgnet.add(block2())
rgnet.add(tf.keras.layers.Dense(1))
rgnet(X)


<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[0.00162046],
       [0.0030088 ]], dtype=float32)>

In [22]:
print(rgnet.summary())

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_6 (Sequential)   (2, 4)                    80        
                                                                 
 dense_20 (Dense)            (2, 1)                    5         
                                                                 
Total params: 85
Trainable params: 85
Non-trainable params: 0
_________________________________________________________________
None


In [21]:
rgnet.layers[0].layers[1].layers[1].weights[1]

<tf.Variable 'dense_17/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>

## Parameter Initialization

## Built-in Initialization

In [23]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4, activation=tf.nn.relu,
        kernel_initializer=tf.random_normal_initializer(mean=0, stddev=0.01),
        bias_initializer=tf.zeros_initializer()),
    tf.keras.layers.Dense(1)])

net(X)
net.weights[0], net.weights[1]

(<tf.Variable 'dense_21/kernel:0' shape=(4, 4) dtype=float32, numpy=
 array([[-0.02311034,  0.00541767,  0.0009859 ,  0.00529982],
        [-0.00835866, -0.01020935,  0.00703625, -0.01651024],
        [-0.00710256, -0.00829015, -0.00594791,  0.006832  ],
        [ 0.00503142, -0.03206465,  0.00380568,  0.00037856]],
       dtype=float32)>,
 <tf.Variable 'dense_21/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>)

In [24]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4, activation=tf.nn.relu,
        kernel_initializer=tf.keras.initializers.Constant(1),
        bias_initializer=tf.zeros_initializer()),
    tf.keras.layers.Dense(1),
])

net(X)
net.weights[0], net.weights[1]


(<tf.Variable 'dense_23/kernel:0' shape=(4, 4) dtype=float32, numpy=
 array([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]], dtype=float32)>,
 <tf.Variable 'dense_23/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>)

In [25]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4,
        activation=tf.nn.relu,
        kernel_initializer=tf.keras.initializers.GlorotUniform()),
    tf.keras.layers.Dense(
        1, kernel_initializer=tf.keras.initializers.Constant(42)),
])

net(X)
print(net.layers[1].weights[0])
print(net.layers[2].weights[0])


<tf.Variable 'dense_25/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[ 0.01289243,  0.6785955 ,  0.33759826,  0.2503119 ],
       [-0.71570385, -0.6432645 , -0.12552392,  0.3826905 ],
       [-0.12250108, -0.4539554 ,  0.0009923 , -0.5817836 ],
       [-0.06155252,  0.3749178 ,  0.10388124,  0.06562257]],
      dtype=float32)>
<tf.Variable 'dense_26/kernel:0' shape=(4, 1) dtype=float32, numpy=
array([[42.],
       [42.],
       [42.],
       [42.]], dtype=float32)>


## Custom Initialization

In [26]:
class MyInit(tf.keras.initializers.Initializer):
    def __call__(self, shape, dtype=None):
        data=tf.random.uniform(shape, -10, 10, dtype=dtype)
        factor=(tf.abs(data) >= 5)
        factor=tf.cast(factor, tf.float32)
        return data * factor

net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4,
        activation=tf.nn.relu,
        kernel_initializer=MyInit()),
    tf.keras.layers.Dense(1),
])

net(X)
print(net.layers[1].weights[0])


<tf.Variable 'dense_27/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[-0.       ,  8.917295 ,  0.       ,  5.4361725],
       [ 0.       ,  9.136724 , -7.4342513, -5.5759788],
       [ 7.074051 , -0.       , -8.688623 , -0.       ],
       [-6.7189646,  8.248459 ,  0.       ,  0.       ]], dtype=float32)>


In [27]:
net.layers[1].weights[0][:].assign(net.layers[1].weights[0] + 1)
net.layers[1].weights[0][0, 0].assign(42)
net.layers[1].weights[0]


<tf.Variable 'dense_27/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[42.       ,  9.917295 ,  1.       ,  6.4361725],
       [ 1.       , 10.136724 , -6.4342513, -4.5759788],
       [ 8.074051 ,  1.       , -7.6886234,  1.       ],
       [-5.7189646,  9.248459 ,  1.       ,  1.       ]], dtype=float32)>

## Tied Parameters

In [30]:
shared = tf.keras.layers.Dense(4, activation=tf.nn.relu)
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    shared,
    shared,
    tf.keras.layers.Dense(1),
])

net(X)

print(len(net.layers) == 3)

True


# Deferred Initialization

## Instantiating a Network

In [31]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation=tf.nn.relu),
    tf.keras.layers.Dense(10),
])

In [32]:
[net.layers[i].get_weights() for i in range(len(net.layers))]

[[], []]

In [33]:
X = tf.random.uniform((2, 20))
net(X)
[w.shape for w in net.get_weights()]

[(20, 256), (256,), (256, 10), (10,)]

# Custom Layers

## Layers without Parameters

In [34]:
class CenteredLayer(tf.keras.Model):
    def __init__(self):
        super().__init__()

    def call(self, inputs):
        return inputs - tf.reduce_mean(inputs)

In [35]:
layer = CenteredLayer()
layer(tf.constant([1, 2, 3, 4, 5]))

<tf.Tensor: shape=(5,), dtype=int32, numpy=array([-2, -1,  0,  1,  2], dtype=int32)>

In [36]:
net = tf.keras.Sequential([tf.keras.layers.Dense(128), CenteredLayer()])

In [37]:
Y = net(tf.random.uniform((4, 8)))
tf.reduce_mean(Y)

<tf.Tensor: shape=(), dtype=float32, numpy=-7.450581e-09>

##  Layers with Parameters

In [38]:
class MyDense(tf.keras.Model):
    def __init__(self, units):
        super().__init__()
        self.units = units

    def build(self, X_shape):
        self.weight = self.add_weight(name='weight',
            shape=[X_shape[-1], self.units],
            initializer=tf.random_normal_initializer())
        self.bias = self.add_weight(
            name='bias', shape=[self.units],
            initializer=tf.zeros_initializer())

    def call(self, X):
        linear = tf.matmul(X, self.weight) + self.bias
        return tf.nn.relu(linear)

In [39]:
dense = MyDense(3)
dense(tf.random.uniform((2, 5)))
dense.get_weights()

[array([[ 0.00339859,  0.05730543,  0.07383379],
        [-0.032985  , -0.04812272, -0.03787738],
        [ 0.00326856,  0.03924185, -0.01200323],
        [-0.12624119,  0.00693509,  0.00111621],
        [-0.04056213,  0.00377347,  0.02723102]], dtype=float32),
 array([0., 0., 0.], dtype=float32)]

In [40]:
dense(tf.random.uniform((2, 5)))

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[0.        , 0.01475263, 0.00833302],
       [0.        , 0.03058792, 0.02678737]], dtype=float32)>

In [41]:
net = tf.keras.models.Sequential([MyDense(8), MyDense(1)])
net(tf.random.uniform((2, 64)))

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[0.        ],
       [0.02381007]], dtype=float32)>

# File I/O

## Loading and Saving Tensors

In [42]:
import numpy as np

In [43]:
x = tf.range(4)
np.save('x-file.npy', x)

In [45]:
x2 = np.load('x-file.npy', allow_pickle=True)
x2

array([0, 1, 2, 3], dtype=int32)

In [44]:
y = tf.zeros(4)
np.save('xy-files.npy', [x, y])
x2, y2 = np.load('xy-files.npy', allow_pickle=True)
(x2, y2)

(array([0., 1., 2., 3.]), array([0., 0., 0., 0.]))

In [46]:
mydict = {'x': x, 'y': y}
np.save('mydict.npy', mydict)
mydict2 = np.load('mydict.npy', allow_pickle=True)
mydict2

array({'x': <tf.Tensor: shape=(4,), dtype=int32, numpy=array([0, 1, 2, 3], dtype=int32)>, 'y': <tf.Tensor: shape=(4,), dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>},
      dtype=object)

## Loading and Saving Model Parameters

In [47]:
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.hidden = tf.keras.layers.Dense(units=256, activation=tf.nn.relu)
        self.out = tf.keras.layers.Dense(units=10)

    def call(self, inputs):
        x = self.flatten(inputs)
        x = self.hidden(x)
        return self.out(x)

net = MLP()
X = tf.random.uniform((2, 20))
Y = net(X)

In [49]:
net.save_weights('mlp.params')

In [50]:
clone = MLP()
clone.load_weights('mlp.params')

In [51]:
Y_clone = clone(X)
Y_clone == Y

<tf.Tensor: shape=(2, 10), dtype=bool, numpy=
array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True]])>

# GPUs

## Computing Devices

In [52]:
tf.device('/CPU:0'), tf.device('/GPU:0'), tf.device('/GPU:1')

(<tensorflow.python.eager.context._EagerDeviceContext at 0x7f3a81ca75a0>,
 <tensorflow.python.eager.context._EagerDeviceContext at 0x7f3a81da83c0>)

In [53]:
len(tf.config.experimental.list_physical_devices('GPU'))

0

In [54]:
def try_gpu(i=0):
    """Return gpu(i) if exists, otherwise return cpu()."""
    if len(tf.config.experimental.list_physical_devices('GPU')) >= i + 1:
        return tf.device(f'/GPU:{i}')
    return tf.device('/CPU:0')

def try_all_gpus():
    """Return all available GPUs, or [cpu(),] if no GPU exists."""
    num_gpus = len(tf.config.experimental.list_physical_devices('GPU'))
    devices = [tf.device(f'/GPU:{i}') for i in range(num_gpus)]
    return devices if devices else [tf.device('/CPU:0')]

try_gpu(), try_gpu(10), try_all_gpus()

(<tensorflow.python.eager.context._EagerDeviceContext at 0x7f3a81caa3c0>,
 [<tensorflow.python.eager.context._EagerDeviceContext at 0x7f3a81caa0a0>])

## Tensors and GPUs

In [55]:
x = tf.constant([1, 2, 3])
x.device

'/job:localhost/replica:0/task:0/device:CPU:0'

### Storage on the GPU

In [56]:
with try_gpu():
    X = tf.ones((2, 3))
X

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 1., 1.],
       [1., 1., 1.]], dtype=float32)>

In [57]:
with try_gpu(1):
    Y = tf.random.uniform((2, 3))
Y

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[0.10904408, 0.09428704, 0.8986889 ],
       [0.24802876, 0.20739043, 0.3113098 ]], dtype=float32)>

### Copying

In [ ]:
with try_gpu(1):
    Z = X
print(X)
print(Z)

tf.Tensor(
[[1. 1. 1.]
 [1. 1. 1.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[1. 1. 1.]
 [1. 1. 1.]], shape=(2, 3), dtype=float32)


In [ ]:
Y + Z

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1.989082 , 1.03084  , 1.000384 ],
       [1.1207649, 1.0950406, 1.6928184]], dtype=float32)>

In [ ]:
with try_gpu(1):
    Z2 = Z
Z2 is Z

True

## Neural Networks and GPUs

In [58]:
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    net = tf.keras.models.Sequential([
        tf.keras.layers.Dense(1)])

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


In [59]:
net(X)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[-2.3378246],
       [-2.3378246]], dtype=float32)>

In [60]:
net.layers[0].weights[0].device, net.layers[0].weights[1].device

('/job:localhost/replica:0/task:0/device:CPU:0',
 '/job:localhost/replica:0/task:0/device:CPU:0')